# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 17 2022 1:17PM,237417,19,ACG-2102490100,ACG North America LLC,Released
1,May 17 2022 1:14PM,237416,19,ACG-2102490098,ACG North America LLC,Released
2,May 17 2022 1:13PM,237415,10,SNAP4500026139-BO,"Snap Medical Industries, LLC",Released
3,May 17 2022 1:10PM,237414,10,SNAP4500026137,"Snap Medical Industries, LLC",Released
4,May 17 2022 1:10PM,237414,10,SNAP4500026138-BO,"Snap Medical Industries, LLC",Released
5,May 17 2022 12:58PM,237413,20,SSNA-2173612679,Sartorius Stedim North America,Released
6,May 17 2022 12:40PM,237411,10,Enova-9978,Emerginnova,Released
7,May 17 2022 12:18PM,237406,10,MET210200,"Methapharm, Inc.",Released
8,May 17 2022 12:18PM,237406,10,MET210201,"Methapharm, Inc.",Executing
9,May 17 2022 12:18PM,237406,10,MET210215,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,237413,Released,1
39,237414,Released,2
40,237415,Released,1
41,237416,Released,1
42,237417,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237413,NaN,NaN,1.0
237414,NaN,NaN,2.0
237415,NaN,NaN,1.0
237416,NaN,NaN,1.0
237417,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0.0,1.0,0.0
237311,0.0,0.0,6.0
237319,0.0,0.0,5.0
237323,0.0,1.0,0.0
237326,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237023,0,1,0
237311,0,0,6
237319,0,0,5
237323,0,1,0
237326,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,0,1,0
1,237311,0,0,6
2,237319,0,0,5
3,237323,0,1,0
4,237326,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237023,,1,
1,237311,,,6
2,237319,,,5
3,237323,,1,
4,237326,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 17 2022 1:17PM,237417,19,ACG North America LLC
1,May 17 2022 1:14PM,237416,19,ACG North America LLC
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC"
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC"
5,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America
6,May 17 2022 12:40PM,237411,10,Emerginnova
7,May 17 2022 12:18PM,237406,10,"Methapharm, Inc."
20,May 17 2022 12:11PM,237408,10,Eywa Pharma Inc.
21,May 17 2022 12:10PM,237407,19,"GCH Granules USA, Inc."
22,May 17 2022 12:05PM,237405,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 17 2022 1:17PM,237417,19,ACG North America LLC,,,1
1,May 17 2022 1:14PM,237416,19,ACG North America LLC,,,1
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC",,,1
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC",,,2
4,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America,,,1
5,May 17 2022 12:40PM,237411,10,Emerginnova,,,1
6,May 17 2022 12:18PM,237406,10,"Methapharm, Inc.",,6,7
7,May 17 2022 12:11PM,237408,10,Eywa Pharma Inc.,,,1
8,May 17 2022 12:10PM,237407,19,"GCH Granules USA, Inc.",,1,
9,May 17 2022 12:05PM,237405,20,"ACI Healthcare USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 1:17PM,237417,19,ACG North America LLC,1,,
1,May 17 2022 1:14PM,237416,19,ACG North America LLC,1,,
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC",1,,
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC",2,,
4,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America,1,,
5,May 17 2022 12:40PM,237411,10,Emerginnova,1,,
6,May 17 2022 12:18PM,237406,10,"Methapharm, Inc.",7,6,
7,May 17 2022 12:11PM,237408,10,Eywa Pharma Inc.,1,,
8,May 17 2022 12:10PM,237407,19,"GCH Granules USA, Inc.",,1,
9,May 17 2022 12:05PM,237405,20,"ACI Healthcare USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 1:17PM,237417,19,ACG North America LLC,1,,
1,May 17 2022 1:14PM,237416,19,ACG North America LLC,1,,
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC",1,,
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC",2,,
4,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 1:17PM,237417,19,ACG North America LLC,1.0,NaN,NaN
1,May 17 2022 1:14PM,237416,19,ACG North America LLC,1.0,NaN,NaN
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
4,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 17 2022 1:17PM,237417,19,ACG North America LLC,1.0,0.0,0.0
1,May 17 2022 1:14PM,237416,19,ACG North America LLC,1.0,0.0,0.0
2,May 17 2022 1:13PM,237415,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,May 17 2022 1:10PM,237414,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
4,May 17 2022 12:58PM,237413,20,Sartorius Stedim North America,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3560845,138.0,9.0,2.0
12,237342,0.0,1.0,0.0
15,474724,21.0,5.0,0.0
16,1424216,6.0,0.0,0.0
18,474346,0.0,2.0,0.0
19,712240,2.0,1.0,0.0
20,2373681,58.0,0.0,0.0
21,237381,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3560845,138.0,9.0,2.0
1,12,237342,0.0,1.0,0.0
2,15,474724,21.0,5.0,0.0
3,16,1424216,6.0,0.0,0.0
4,18,474346,0.0,2.0,0.0
5,19,712240,2.0,1.0,0.0
6,20,2373681,58.0,0.0,0.0
7,21,237381,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,138.0,9.0,2.0
1,12,0.0,1.0,0.0
2,15,21.0,5.0,0.0
3,16,6.0,0.0,0.0
4,18,0.0,2.0,0.0
5,19,2.0,1.0,0.0
6,20,58.0,0.0,0.0
7,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,12,Released,0.0
2,15,Released,21.0
3,16,Released,6.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,1.0,5.0,0.0,2.0,1.0,0.0,1.0
Released,138.0,0.0,21.0,6.0,0.0,2.0,58.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,1.0,5.0,0.0,2.0,1.0,0.0,1.0
2,Released,138.0,0.0,21.0,6.0,0.0,2.0,58.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,1.0,5.0,0.0,2.0,1.0,0.0,1.0
2,Released,138.0,0.0,21.0,6.0,0.0,2.0,58.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()